## New Standardization Workflow

In [6]:
import pandas as pd

df = pd.read_csv('NW Publisher – Address Export - 20250722 - complete.csv')

# Filter and clean
clean = df[df['Status'] != 'NewFromPublisher']

# Deduplicate by Number + Street + PostalCode
clean_unique = clean.drop_duplicates(subset=['Number','Street','PostalCode'], keep='first')

# Save
out = 'cleaned_addresses.csv'
clean_unique.to_csv(out, index=False)
out


'cleaned_addresses.csv'

In [7]:
import os
import time
import json
import requests
import pandas as pd
from datetime import datetime

# --- CONFIGURE ---
AUTH_ID = "c158cf93-8a4d-c43c-dade-f372cbeac850"   # replace if needed
AUTH_TOKEN = "e7UMwEf0GevqQpxwdLaO"                # replace if needed
SMARTY_URL = "https://us-street.api.smarty.com/street-address"

INPUT_CSV = os.path.join("cleaned_sample.csv")  # input file (expects Number/Street/Suburb/State/PostalCode or similar)
CACHE_DIR = os.path.join("json_cache")
os.makedirs(CACHE_DIR, exist_ok=True)

# candidate names for unit/apartment in input
SECONDARY_CANDIDATES = [
    "ApartmentNumber", "Apartment", "Apt", "Unit", "Secondary", "SecondaryUnit",
    "apt_number", "apt", "unit_number"
]

def get_input_secondary(row):
    for k in SECONDARY_CANDIDATES:
        if k in row.index:
            v = row.get(k)
            if pd.notna(v) and str(v).strip():
                return str(v).strip()
    return ""

def safe_filename(s):
    return "".join(c if c.isalnum() or c in "._- " else "_" for c in str(s)).strip().replace(" ", "_")

def save_response(name_base, data, meta):
    path = os.path.join(CACHE_DIR, f"{name_base}.json")
    meta_path = os.path.join(CACHE_DIR, f"{name_base}_meta.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2)
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)

def main():
    df = pd.read_csv(INPUT_CSV)
    for i, row in df.iterrows():
        number = str(row.get("Number", "")).strip()
        street_raw = str(row.get("Street", "")).strip()
        street = f"{number} {street_raw}".strip()
        city = str(row.get("Suburb", "")).strip()
        state = str(row.get("State", "CA")).strip()
        zipcode = str(row.get("PostalCode", "")).replace(".0", "").strip()
        input_secondary = get_input_secondary(row)

        params = {
            "auth-id": AUTH_ID,
            "auth-token": AUTH_TOKEN,
            "street": street,
            "city": city,
            "state": state,
            "zipcode": zipcode,
        }
        if input_secondary:
            params["secondary"] = input_secondary

        # best-effort filename check: if response cached by input row (index) skip
        tentative_name = f"row_{i}"
        meta_stub = {"row_index": int(i), "sent_params": params, "input_secondary": input_secondary, "timestamp": datetime.utcnow().isoformat() + "Z"}

        # Do request
        try:
            r = requests.get(SMARTY_URL, params=params, timeout=15)
            # try parse JSON even on non-200 to capture error responses in cache
            data = r.json() if r.content else None
            # compute name base from delivery_point_barcode if possible
            name_base = tentative_name
            if isinstance(data, list) and len(data) > 0 and isinstance(data[0], dict):
                dpb = data[0].get("delivery_point_barcode") or data[0].get("metadata", {}).get("delivery_point_barcode")
                if dpb:
                    name_base = safe_filename(dpb)
                else:
                    # try primary number + zipcode for nicer name
                    comps = data[0].get("components", {})
                    pn = comps.get("primary_number") or number or str(i)
                    z = comps.get("zipcode") or zipcode or ""
                    name_base = safe_filename(f"{pn}_{z}_{i}")
            else:
                # fallback name includes street and index
                name_base = safe_filename(f"{street_raw}_{i}")[:120] or tentative_name

            save_response(name_base, data, meta_stub | {"status_code": r.status_code})
            print(f"[{i}] saved cache: {name_base}.json (HTTP {r.status_code})")

            # Respect rate limit a bit
            time.sleep(0.5)

        except Exception as e:
            # save error meta for debugging
            err_meta = meta_stub | {"error": str(e)}
            err_name = safe_filename(f"error_row_{i}")
            with open(os.path.join(CACHE_DIR, f"{err_name}_meta.json"), "w", encoding="utf-8") as f:
                json.dump(err_meta, f, indent=2)
            print(f"[{i}] Exception: {e}")

if __name__ == "__main__":
    main()

[0] saved cache: 903052323120.json (HTTP 200)
[1] saved cache: 903052323166.json (HTTP 200)
[2] saved cache: 903052346170.json (HTTP 200)
[3] saved cache: 903052346215.json (HTTP 200)


In [1]:
import os
import json
import pandas as pd
from glob import glob

CACHE_DIR = os.path.join("json_cache")
OUT_CSV = os.path.join("output", "cached_parsed_addresses.csv")
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)

def parse_file(path):
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except Exception:
        return None

    # If data is a list, take first candidate (common Smarty response)
    entry = data[0] if isinstance(data, list) and len(data) > 0 else (data if isinstance(data, dict) else None)
    if not entry:
        return None

    comps = entry.get("components", {})
    meta = entry.get("metadata", {})
    analysis = entry.get("analysis", {})
    row = {
        "cache_file": os.path.basename(path),
        "delivery_line_1": entry.get("delivery_line_1", ""),
        "delivery_line_2": entry.get("delivery_line_2", ""),
        "last_line": entry.get("last_line", ""),
        "delivery_point_barcode": entry.get("delivery_point_barcode", "") or meta.get("delivery_point_barcode", ""),
        # components
        "primary_number": comps.get("primary_number", ""),
        "street_predirection": comps.get("street_predirection", ""),
        "street_name": comps.get("street_name", ""),
        "street_suffix": comps.get("street_suffix", ""),
        "secondary_designator": comps.get("secondary_designator", ""),
        "secondary_number": comps.get("secondary_number", ""),
        "city_name": comps.get("city_name", "") or comps.get("default_city_name", ""),
        "state_abbreviation": comps.get("state_abbreviation", ""),
        "zipcode": comps.get("zipcode", ""),
        "plus4_code": comps.get("plus4_code", ""),
        "delivery_point": comps.get("delivery_point", ""),
        "delivery_point_check_digit": comps.get("delivery_point_check_digit", ""),
        
        # metadata/location
        "latitude": meta.get("latitude", ""),
        "longitude": meta.get("longitude", ""),
        "precision": meta.get("precision", ""),
        # analysis/dpv
        "dpv_match_code": analysis.get("dpv_match_code", ""),
        "dpv_footnotes": analysis.get("dpv_footnotes", ""),
        "active": analysis.get("active", ""),
    }
    return row

def main():
    files = sorted(glob(os.path.join(CACHE_DIR, "*.json")))
    # ignore meta companion files if any (we included *_meta.json)
    files = [f for f in files if not f.endswith("_meta.json")]
    rows = []
    for fpath in files:
        parsed = parse_file(fpath)
        if parsed:
            rows.append(parsed)
    if not rows:
        print("No parsed rows found in cache.")
        return
    df = pd.DataFrame(rows)
    # add a composed StandardizedAddress column using delivery lines when available
    def compose_std(r):
        parts = []
        if r.get("delivery_line_1"):
            parts.append(r["delivery_line_1"])
        if r.get("delivery_line_2"):
            parts.append(r["delivery_line_2"])
        if r.get("last_line"):
            parts.append(r["last_line"])
        return ", ".join([p for p in parts if p])
    df["StandardizedAddress"] = df.apply(compose_std, axis=1)
    df.to_csv(OUT_CSV, index=False)
    print("Saved parsed cached data to:", OUT_CSV)

if __name__ == "__main__":
    main()

Saved parsed cached data to: output\cached_parsed_addresses.csv
